In [ ]:
import numpy as np

In [ ]:
#PCA数据降维
def PCA(matX,k):
    average=np.mean(matX,axis=0)
    m,n=matX.shape
    avgs=np.tile(average,(m,1))
    data_adjust=matX-avgs
    data_adjust=np.array(data_adjust,dtype=float)
    covX=np.cov(data_adjust.T)
    feaValue,feaVec=np.linalg.eig(covX)
    index=np.argsort(-feaValue)
    selectVec=np.matrix(feaVec.T[index[:k]])
    final=matX*selectVec.T
    return(final)

In [ ]:
from PIL import Image

In [ ]:
im=Image.open('C:\\Users\\zhou\\Desktop\\play0.jpg')

In [ ]:
r,g,b=im.split()

In [ ]:
r=np.array(r)

In [ ]:
r1=PCA(r,28)
r2=PCA(r1.T,28)
r2=r2.T
r=(r2.real-r2.real.min(0))/(r2.real.max(0)-r2.real.min(0))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

In [ ]:
r=torch.tensor(r)
r=Variable(r).float()
r=r.reshape(1,1,28,28)

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.conv1=nn.Conv2d(1,10,kernel_size=5)
        self.conv2=nn.Conv2d(10,20,kernel_size=5)
        self.conv2_drop=nn.Dropout2d()
        self.fc1=nn.Linear(320,50)
        self.fc2=nn.Linear(50,10)
    def forward(self,x):
        x=F.relu(F.max_pool2d(self.conv1(x),2))
        x=F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        x=x.view(-1,320)
        x=F.relu(self.fc1(x))
        x=F.dropout(x,training=self.training)
        x=self.fc2(x)
        return F.log_softmax(x,dim=1)

In [ ]:
model=LeNet()
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.005,momentum=0.9)

In [ ]:
torch.cuda

In [ ]:
cuda_gpu=torch.cuda.is_available

In [ ]:
from torchvision import datasets,transforms
batch_num_size=64
train_loader=torch.utils.data.DataLoader(datasets.MNIST('data',train=True,download=True,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])),batch_size=batch_num_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(datasets.MNIST('data',train=False,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])),batch_size=batch_num_size,shuffle=True)

In [ ]:
def test(model,epoch,criterion,data_loader):
    model.eval()
    test_loss=0
    correct=0
    for data,target in data_loader:
        #if cuda_gpu:
            #data,target=data.cuda(),target.cuda()
           # model.cuda()
        data,target=Variable(data),Variable(target)
        output=model(data)
        test_loss+=criterion(output,target).item()
        pred=output.data.max(1)[1]
        correct+=pred.eq(target.data).cpu().sum()
    test_loss/=len(data_loader)
    acc=correct/len(data_loader.dataset)
    return (acc,test_loss),output.data.max(1)[1]

In [ ]:
def train(model,epoch,criterion,optimizer,data_loader):
    model.train()
    for batch_idx,(data,target) in enumerate(data_loader):
        #if cuda_gpu:
           # data,target=data.cuda(),target.cuda()
           # model.cuda()
        data,target=Variable(data),Variable(target)
        output=model(data)
        optimizer.zero_grad()
        print(target)
        loss=criterion(output,target)
        loss.backward()
        optimizer.step()

In [ ]:
epochs=5

In [ ]:
list1=[]
for i,j in train_loader:
    list1.append((i,j))

In [ ]:
for i,j in train_loader:
    print(i,j)
    print('aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa')

In [ ]:
train(model,5,criterion,optimizer,train_loader)

In [ ]:
test(model,epochs,criterion,test_loader) 

In [ ]:
model(r).data.max(1)[1]